In [ ]:
import glob
import json
import os

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage.io as io
from pycocotools.coco import COCO

## #0 Look at the labels

In [ ]:
json_path = '/root/data/small_pen_data_collection/body_parts_keypoints_experiment_labeled.json'

In [ ]:
# # coco fix
# cocofile = json.load(open(json_path))

# images = glob.glob('/root/data/small_pen_data_collection/*_rectified/*.jpg')
# image_dict = {}
# for img in images:
#     image_dict[os.path.basename(img)] = img
    
# for img in cocofile:
#     img_name = img['Labeled Data'].split('%2F')[-1].split('?')[0]
#     img['local_path'] = image_dict[img_name]
    
# with open(json_path, 'w') as f:
#     json.dump(cocofile, f)

In [ ]:
image_data = json.load(open(json_path))[0]

In [ ]:
json.load(open(json_path))

In [ ]:
image = io.imread(image_data['local_path'])[1300:2580, :]
# image = cv2.resize(image, (192, 64))
xratio = 192 / 4096 
yratio = 64 / 3000
f ,ax = plt.subplots(1, figsize=(20, 20))
ax.imshow(image); 

for (class_name, point) in image_data['Label'].items():
    print(class_name)
    if class_name == 'Mouth ':
        continue
    loc = point[0]['geometry']
    # print(loc)
    # print(loc['x'], loc['y']-1300)
    print(loc['x'], loc['y']-1300)
    circle = patches.Circle((loc['x'], loc['y']-1300), radius=10, color='r')
    ax.add_patch(circle)
    # ax.text(loc['x'], loc['y']-10, class_name, fontsize=16, color='w')
# plt.axis('off')
plt.show()

In [ ]:
image_data['Label']

In [ ]:
image_data

## #1 First approach - images chunks

In [ ]:
import cv2
import numpy as np

In [ ]:
# focus on eye detection first

In [ ]:
fish_only = io.imread(image_data['local_path'])[1300:2580, :]
labels = np.zeros((fish_only.shape[0], fish_only.shape[1]))
c = 1
for (class_name, point) in image_data['Label'].items():
    if class_name != 'Eye Center':
        continue
    print(class_name, c)
    loc = point[0]['geometry']
    labels[loc['y']-1300, loc['x']] = c
    c += 1

In [ ]:
nclasses = 1

In [ ]:
def generator(image, labels, batch_size=32):
    # image_resize = cv2.resize(image, (192, 64))
    im = image
    ssize = 64
    tiles = [(im[x:x+ssize, y:y+ssize], labels[x:x+ssize, y:y+ssize]) for x in range(0,im.shape[0], ssize) for y in range(0,im.shape[1], ssize)]
    dic = {}
    for tile in tiles:
        if np.sum(tile[1]) not in dic:
            dic[np.sum(tile[1])] = []
        dic[np.sum(tile[1])].append(tile)
    while True:
        x_batch = np.zeros((batch_size, ssize, ssize, 3))
        y_batch_class = np.zeros((batch_size, 2)) # class + dx + dx
        y_batch_reg = np.zeros((batch_size, 2))
        for i in range(batch_size):
            rk = np.random.choice(list(dic.keys()))
            random_tuple = dic[rk][np.random.randint(0, len(dic[rk]))]
            random_tile = random_tuple[0]
            random_reg = random_tuple[1]
            random_class = np.sum(random_reg)
            x_batch[i,...] = random_tile
            y_batch_class[i, int(random_class)] = 1
            if random_class > 0:
                y_batch_reg[i, 0] = np.nonzero(random_reg)[0][0]
                y_batch_reg[i, 1] = np.nonzero(random_reg)[1][0]
        yield x_batch, {"class": y_batch_class, "reg": y_batch_reg}
        
    
    
    

In [ ]:
gen = generator(fish_only, labels)

In [ ]:
xb, yb = next(gen)

In [ ]:
yb["class"].shape

In [ ]:
yb["reg"].shape

In [ ]:
import keras
from keras.layers import Conv2D, Flatten, Dense, Input, MaxPool2D, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam 

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [ ]:
inp = Input(shape=(64, 64, 3))
x = Conv2D(filters=64, kernel_size=(3, 3))(inp)
x = Conv2D(filters=64, kernel_size=(3, 3))(x)
x = Conv2D(filters=64, kernel_size=(3, 3))(x)
x = MaxPool2D()(x)
x = BatchNormalization()(x)

x = Conv2D(filters=128, kernel_size=(3, 3))(x)
x = Conv2D(filters=128, kernel_size=(3, 3))(x)
x = Conv2D(filters=128, kernel_size=(3, 3))(x)
x = MaxPool2D()(x)
x = BatchNormalization()(x)

x = Flatten()(x)
x = Dense(128)(x)
x = BatchNormalization()(x)
x = Dense(256)(x)
x = BatchNormalization()(x)

classifer = Dense(2, activation="softmax", name="class")(x)
regressor = Dense(2, activation="linear", name="reg")(x)

model = Model(inputs=[inp], outputs=[classifer, regressor])
print(model.summary())

In [ ]:
model.compile(optimizer="adam", loss={"class": "categorical_crossentropy", "reg": "mean_absolute_error"})

In [ ]:
model.fit_generator(gen,steps_per_epoch=10, epochs=100)

# predict test

In [ ]:
eye = xb[0, ...].squeeze().astype(dtype=np.uint8)
plt.imshow(eye)
plt.show()

In [ ]:
# %matplotlib notebook 

In [ ]:
R = model.predict_on_batch(np.expand_dims(xb[0, ...], axis=0))

In [ ]:
R[1]

In [ ]:
im = fish_only
ssize = 32
tiles = [(im[x:x+ssize, y:y+ssize], labels[x:x+ssize, y:y+ssize], (x,y)) for x in range(0,im.shape[0], ssize) for y in range(0,im.shape[1], ssize)]

In [ ]:
for t in tiles:
    if np.sum(t[1]) > 0:
        print(t[2])

In [ ]:
f, ax = plt.subplots(figsize=(20, 10))
ax.imshow(fish_only[200:800, : 800])
# ax.margins(1000)
circle = patches.Circle((414, 529-200), radius=10, color='r')
ax.add_patch(circle)
circle = patches.Circle((384+R[1].squeeze()[0], 512+R[1].squeeze()[1]-200), radius=10, color='b')
ax.add_patch(circle)
plt.show()

In [ ]:
R